In [1]:
import os
import datetime
import json
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu
from src.me_to_neurazi import me_to_neurazi

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())

In [2]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","650.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","653.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("stran") >= 30)

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["100_a","245_a","245_p"], keep="first")
print(len(df))

795736
506565


## Kdy se vyplnovalo pole 655_a?

In [4]:
df.filter(~pl.col("655_a").is_null()).group_by("rok").len().sort(by="rok").join(
    df.group_by('rok').len(), on='rok', how='left'
).with_columns((pl.col("len") / pl.col("len_right")).alias('vyplnenost')).tail(50)

rok,len,len_right,vyplnenost
i64,u32,u32,f64
1976,623,2825,0.220531
1977,669,2892,0.231328
1978,710,2935,0.241908
1979,901,3218,0.279988
1980,1037,3321,0.312255
1981,1036,3164,0.327434
1982,1449,3380,0.428698
1983,2483,3884,0.639289
1984,2523,3989,0.632489


In [5]:
df = df.filter(pl.col("rok").is_between(1989,2024))

In [6]:
df.filter(pl.col("245_a").str.contains("Každý den je nový")).explode("655_a")

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,653_a,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Mandžuková, Jarmila,""","""jn20001103350""","[""aut""]","""1959-""",null,null,null,null,"""nkc20183060546""",""" nam a22 i 4500""","""181206s2018 xr g 0…","""(vázáno) :""","[""Kč 229,00""]","[""978-80-7601-047-5""]",null,null,null,null,null,null,"""1""","""0""","""Každý den je nový začátek""","""každý nový den je zázrak, aneb…","""Jarmila Mandžuková""",null,null,null,null,null,"[""119 stran ;""]",null,"[""14 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","""citáty""","[""fd131832"", ""fd131784"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2018,119,"""pevná"""
"""1""","""Mandžuková, Jarmila,""","""jn20001103350""","[""aut""]","""1959-""",null,null,null,null,"""nkc20183060546""",""" nam a22 i 4500""","""181206s2018 xr g 0…","""(vázáno) :""","[""Kč 229,00""]","[""978-80-7601-047-5""]",null,null,null,null,null,null,"""1""","""0""","""Každý den je nový začátek""","""každý nový den je zázrak, aneb…","""Jarmila Mandžuková""",null,null,null,null,null,"[""119 stran ;""]",null,"[""14 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","""aforismy""","[""fd131832"", ""fd131784"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2018,119,"""pevná"""
"""1""","""Mandžuková, Jarmila,""","""jn20001103350""","[""aut""]","""1959-""",null,null,null,null,"""nkc20183060546""",""" nam a22 i 4500""","""181206s2018 xr g 0…","""(vázáno) :""","[""Kč 229,00""]","[""978-80-7601-047-5""]",null,null,null,null,null,null,"""1""","""0""","""Každý den je nový začátek""","""každý nový den je zázrak, aneb…","""Jarmila Mandžuková""",null,null,null,null,null,"[""119 stran ;""]",null,"[""14 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","""quotations""","[""fd131832"", ""fd131784"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2018,119,"""pevná"""
"""1""","""Mandžuková, Jarmila,""","""jn20001103350""","[""aut""]","""1959-""",null,null,null,null,"""nkc20183060546""",""" nam a22 i 4500""","""181206s2018 xr g 0…","""(vázáno) :""","[""Kč 229,00""]","[""978-80-7601-047-5""]",null,null,null,null,null,null,"""1""","""0""","""Každý den je nový začátek""","""každý nový den je zázrak, aneb…","""Jarmila Mandžuková""",null,null,null,null,null,"[""119 stran ;""]",null,"[""14 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","""aphorisms""","[""fd131832"", ""fd131784"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2018,119,"""pevná"""
"""1""

In [7]:
df.filter(pl.col("245_a").str.contains("Pod dekou"))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,653_a,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Thompson, Craig,""","""xx0037934""","[""aut"", ""ill""]","""1975-""",null,null,null,null,"""nkc20051632284""",""" nam a22 a 4500""","""051123s2005 xr a g 0…","""(váz.) :""","[""Kč 590,00""]","[""80-7341-603-4""]",null,null,null,null,null,null,"""1""","""0""","""Pod dekou""","""ilustrovaný román /""","""Craig Thompson ; [z anglického…",null,null,null,null,null,"[""582 s. :""]","[""il. ;""]","[""25 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","[""americké romány"", ""komiksy"", … ""comics""]","[""fd131796"", ""fd131978"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2005,582,"""pevná"""


In [8]:
zebricek = df.explode("655_a").group_by("655_a").len().sort(by="len",descending=True)
zebricek.filter(pl.col('len') >= 200)

655_a,len
str,u32
"""příručky""",32562
"""handbooks and manuals""",24458
"""učebnice vysokých škol""",21487
"""publikace pro děti""",19853
"""monografie""",18122
"""children's literature""",16902
"""monographs""",16491
"""textbooks (higher)""",13582
"""populárně-naučné publikace""",13514


In [9]:
zebricek.filter(pl.col("655_a").str.contains("dívč"))

655_a,len
str,u32
"""dívčí romány""",210


In [10]:
df.sample(20)

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,653_a,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Vedral, Jiří,""","""mzk2003169026""","[""aut""]","""1973-""",null,null,null,null,"""nkc20112176242""",""" nam a22 a 4500""","""110504s2011 xr f e 0…","""(brož.)""",null,"[""978-80-7457-004-9""]",null,null,null,null,null,null,"""1""","""0""","""Švédsko-český lékařský slovník""",null,"""J. Vedral""",null,null,null,null,null,"[""124 s. ;""]",null,"[""21 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","[""dvojjazyčné překladové slovníky"", ""terminologické slovníky"", … ""terminological dictionaries""]","[""fd208746"", ""fd202538"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2011,124,"""brožovaná"""
"""1""","""Vaňková, Pavlína""","""jn20001103671""","[""aut""]",null,null,null,null,null,"""cpk20000973762""",""" cam a22 a 4500""","""010305s2000 xr e 0…","""(váz.)""",null,"[""80-7240-142-4""]",null,null,null,null,null,null,"""1""","""0""","""Francouzština""","""conversation & dictionnaire tc…","""Pavlína Vaňková""",null,null,null,null,null,"[""384 s. ;""]",null,"[""16 cm""]",null,null,null,…,null,"[""7""]","[""příručky""]","[""fd133209""]","[""czenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2000,384,"""pevná"""
"""1""","""Klucký, Dalibor,""","""kup19950000049127""","[""aut""]","""1933-""",null,null,null,null,"""cpk20162806415""",""" nam a22 aa4500""","""940418s1993 xr 0…","""(brož.) :""","[""Kč 52,00 (za obě části)""]","[""80-7067-226-9""]",null,null,null,null,null,null,"""1""","""0""","""Lineární algebra I""",null,"""Dalibor Klucký""",null,null,null,null,null,"[""2 sv. (353 s.) ;""]",null,"[""29 cm""]",null,null,null,…,"[""determinanty"", ""lineární rovnice"", ""vektorové prostory""]","[""7""]","[""učebnice vysokých škol""]","[""fd133772""]","[""czenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1993,353,"""brožovaná"""
"""1""","""Dubin, Marc,""","""xx0002545""","[""aut""]","""1955-""",null,null,null,null,"""cpk20010887930""",""" cam a22 a 4500""","""010409s2001 xr ab e 0…","""(brož.)""",null,"[""80-7202-746-8""]",null,null,null,null,null,null,"""1""","""0""","""Řecko""","""Athény a pevnina /""","""Marc Dubin ; [z anglického ori…",null,null,null,null,null,"[""352 s. :""]","[""barev. il. a mapy ;""]","[""22 cm""]",null,null,null,…,null,"[""7""]","[""průvodce""]","[""fd133154""]","[""czenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2001,352,"""brožovaná"""
"""1""","""Štohl, Pavel,""","""xx0024514""","[""aut""]","""1960-""",null,null,null,null,"""nkc20233554977""",""" nam a22 i 4500""","""231012s2023 xr e p 0…","""(brožováno)""",null,"

In [11]:
vyvoj = df.filter(pl.col("rok").is_between(2000,2003)).explode("655_a").group_by("655_a").len().join(
    df.filter(pl.col("rok").is_between(2021,2024)).explode("655_a").group_by("655_a").len(), on="655_a", how="full"
).with_columns((pl.col("len") / pl.col("len_right")).alias("rozdil")).with_columns((pl.col("len") + pl.col("len_right")).alias("celkem")).sort(by="rozdil")

In [12]:
vyvoj.filter(pl.col("celkem") >= 100).head(150)

655_a,len,655_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
"""women's novels""",1,"""women's novels""",596,0.001678,597
"""romány pro ženy""",1,"""romány pro ženy""",596,0.001678,597
"""fantasy komiksy""",1,"""fantasy komiksy""",259,0.003861,260
"""fantasy comics""",1,"""fantasy comics""",259,0.003861,260
"""akční a dobrodružné komiksy""",1,"""akční a dobrodružné komiksy""",255,0.003922,256
"""action and adventure comics""",1,"""action and adventure comics""",252,0.003968,253
"""superhrdinské komiksy""",1,"""superhrdinské komiksy""",221,0.004525,222
"""superhero comics""",1,"""superhero comics""",219,0.004566,220
"""autorské knihy""",1,"""autorské knihy""",165,0.006061,166


In [13]:
vyvoj.filter(pl.col("celkem") >= 100).tail(150)

655_a,len,655_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
"""Czech prose""",147,"""Czech prose""",429,0.342657,576
"""detective stories""",27,"""detective stories""",78,0.346154,105
"""obrazové publikace""",372,"""obrazové publikace""",1064,0.349624,1436
"""české romány""",751,"""české romány""",2097,0.358131,2848
"""texts""",38,"""texts""",101,0.376238,139
"""catalogs""",66,"""catalogs""",175,0.377143,241
"""popular works""",742,"""popular works""",1966,0.377416,2708
"""fotografické publikace""",556,"""fotografické publikace""",1459,0.381083,2015
"""autobiografické prózy""",44,"""autobiografické prózy""",114,0.385965,158


In [14]:
df.select(pl.col("653_a")).drop_nulls().sample(50)

653_a
list[str]
"[""legislativní hlediska"", ""měnová odluka"", … ""Česká republika""]"
"[""Rakouské Slezsko""]"
"[""jazyková úroveň A1"", ""jazyková úroveň A2""]"
"[""dospívání"", ""literatura pro mládež"", ""mládež""]"
"[""speciální pedagogika raného věku""]"
"[""INFORMIX-4GL"", ""programovací jazyky"", ""programování""]"
"[""ruský formalismus""]"
"[""příručky""]"
"[""jídelní lístek"", ""jídla"", … ""česko-německé slovníky""]"


In [15]:
vyvoj2 = df.filter(pl.col("rok").is_between(2000,2003)).explode("653_a").group_by("653_a").len().join(
    df.filter(pl.col("rok").is_between(2021,2024)).explode("653_a").group_by("653_a").len(), on="653_a", how="full"
).with_columns((pl.col("len") / pl.col("len_right")).alias("rozdil")).fill_null(0).with_columns((pl.col("len") + pl.col("len_right")).alias("celkem")).sort(by="rozdil")
vyvoj2.filter(pl.col("celkem") > 5)

653_a,len,653_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
null,0,"""literatura new adult""",50,0.0,50
null,0,"""jazyková úroveň A1""",15,0.0,15
null,0,"""jazyková úroveň B2""",7,0.0,7
null,0,"""jazyková úroveň A2""",14,0.0,14
null,0,null,32189,0.0,32189
null,0,"""jazyková úroveň B1""",14,0.0,14
null,31258,null,0,0.0,31258
"""denní četba""",13,"""denní četba""",44,0.295455,57


In [16]:
vyvoj3 = df.filter(pl.col("rok").is_between(2000,2003)).explode("655_a").group_by("655_a").len().join(
    df.filter(pl.col("rok").is_between(2021,2024)).explode("655_a").group_by("655_a").len(), on="655_a", how="full"
).with_columns((pl.col("len") / pl.col("len_right")).alias("rozdil")).fill_null(0).with_columns((pl.col("len") + pl.col("len_right")).alias("celkem")).sort(by="rozdil")

In [17]:
vyvoj3.filter((pl.col("len") > 1) & (pl.col('len_right') > 1) & (pl.col("celkem") > 100)).sort(by='rozdil',descending=True)

655_a,len,655_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
"""povídky""",112,"""povídky""",6,18.666667,118
"""milostné povídky""",492,"""milostné povídky""",37,13.297297,529
"""lyrická poezie""",187,"""lyrická poezie""",20,9.35,207
"""lyric poetry""",148,"""lyric poetry""",20,7.4,168
"""právní předpisy""",225,"""právní předpisy""",32,7.03125,257
"""učebnice vysokých škol""",3224,"""učebnice vysokých škol""",536,6.014925,3760
"""legal regulations""",153,"""legal regulations""",29,5.275862,182
"""studie""",2336,"""studie""",484,4.826446,2820
"""vědecko-fantastické povídky""",229,"""vědecko-fantastické povídky""",50,4.58,279


In [18]:
vyvoj3.filter(pl.col("celkem") > 200).filter(pl.col('rozdil') > 1).sort(by='rozdil',descending=True)

655_a,len,655_a_right,len_right,rozdil,celkem
str,u32,str,u32,f64,u32
"""milostné povídky""",492,"""milostné povídky""",37,13.297297,529
"""lyrická poezie""",187,"""lyrická poezie""",20,9.35,207
"""právní předpisy""",225,"""právní předpisy""",32,7.03125,257
"""učebnice vysokých škol""",3224,"""učebnice vysokých škol""",536,6.014925,3760
"""studie""",2336,"""studie""",484,4.826446,2820
"""vědecko-fantastické povídky""",229,"""vědecko-fantastické povídky""",50,4.58,279
"""textbooks (higher)""",2249,"""textbooks (higher)""",533,4.219512,2782
"""učebnice""",392,"""učebnice""",113,3.469027,505
"""dobrodružné povídky""",315,"""dobrodružné povídky""",101,3.118812,416


In [19]:
top = ['komiksy','young adult',"rozhovory","erotické romány"]
flop = ["učebnice","slovníky","encyklopedie","právní předpisy"]

In [20]:
def podil_zanru(zanr, rok=1989):
    return df.filter(pl.col("rok").is_between(rok,2024)).explode("655_a").filter(
        pl.col("655_a").str.contains("(?i)" + zanr)
    ).group_by(
        "rok"
    ).len(
    ).join(df.filter(pl.col("rok") >= rok).drop_nulls(subset=["655_a"]).group_by("rok").len(), on="rok", how="right").with_columns((pl.col("len") / pl.col("len_right")).alias("podil")).sort(by="rok").rename({'len':'pocet'}).with_columns(pl.lit(zanr).alias("zanr"))

In [21]:
df.explode("655_a").filter(pl.col('655_a') == "erotické romány").group_by('245_a').len().sort(by='len',descending=True)

245_a,len
str,u32
"""Vlci zvěrokruhu""",4
"""Hříšné odstíny vášně""",4
"""Jezdci apokalypsy""",4
"""Twisted""",4
"""Královská sága plná sexu""",4
"""Barvy lásky""",4
"""Crossfire""",3
"""Poslední jezdci""",3
"""Mount trilogy""",3


In [22]:
def co_to_je(nazev):
    return df.filter(pl.col("245_a").str.contains(nazev)).select("655_a").explode("655_a").to_series().to_list()

In [23]:
co_to_je("Neztratit se sama sobě")

['rozhovory', 'interviews']

In [24]:
co_to_je("Otevřené hranice")

['naučné komiksy', 'nonfiction comics']

In [25]:
podil_zanru("nonfiction comics")

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
null,1989,3115,null,"""nonfiction comics"""
null,1990,3220,null,"""nonfiction comics"""
null,1991,3598,null,"""nonfiction comics"""
null,1992,4452,null,"""nonfiction comics"""
null,1993,5949,null,"""nonfiction comics"""
null,1994,6067,null,"""nonfiction comics"""
null,1995,6784,null,"""nonfiction comics"""
null,1996,7014,null,"""nonfiction comics"""
null,1997,7395,null,"""nonfiction comics"""


In [26]:
podil_zanru("komiks")

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
4,1989,3115,0.001284,"""komiks"""
6,1990,3220,0.001863,"""komiks"""
8,1991,3598,0.002223,"""komiks"""
8,1992,4452,0.001797,"""komiks"""
5,1993,5949,0.00084,"""komiks"""
11,1994,6067,0.001813,"""komiks"""
8,1995,6784,0.001179,"""komiks"""
6,1996,7014,0.000855,"""komiks"""
10,1997,7395,0.001352,"""komiks"""


In [27]:
top_data = pl.concat([podil_zanru(z, rok=2000) for z in top])
flop_data = pl.concat([podil_zanru(z, rok=2000) for z in flop])

In [28]:
flop_data

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
971,2000,7629,0.127277,"""učebnice"""
979,2001,7757,0.126209,"""učebnice"""
1090,2002,7776,0.140175,"""učebnice"""
1119,2003,8038,0.139214,"""učebnice"""
1065,2004,8477,0.125634,"""učebnice"""
1157,2005,8791,0.131612,"""učebnice"""
1059,2006,9212,0.114959,"""učebnice"""
1198,2007,9446,0.126826,"""učebnice"""
964,2008,9643,0.099969,"""učebnice"""


In [29]:
# flop_data = flop_data.with_columns(pl.col("zanr").replace({"právní předpisy":"práv. předp.","encyklopedie":"encykl."}))

In [30]:
sirka_sloupcu = 0.9

top_graf = alt.Chart(
    alt_friendly(top_data),title=alt.Title(
        "Čeho v Česku vychází víc než v roce 2000…",subtitle="(Každý graf má své vlastní měřítko.)"
    ), width=55, height=kredity['vyska_nizkych']
).mark_bar(color='#70871E', width=sirka_sloupcu).encode(
    alt.X("rok:T", title=None, axis=alt.Axis(tickCount=1)),
    alt.Y("pocet", title=None, axis=alt.Axis(orient='right', domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domainMin=0), ),
    alt.Column("zanr", title=None, sort=top,
           header=alt.Header(labelFont='Asap'))
).resolve_axis(y="independent",x="independent").resolve_scale(y="independent")

flop_graf = alt.Chart(
    alt_friendly(flop_data),title="…a čeho míň", width=48, height=55
).mark_bar(color='#DB842F', width=sirka_sloupcu).encode(
    alt.X("rok:T", title=None, axis=alt.Axis(tickCount=1)),
    alt.Y("pocet", title=None, axis=alt.Axis(orient='right', domainOpacity=0, tickColor='#DCDDD6')),
    alt.Column("zanr", title=None,
           header=alt.Header(labelFont='Asap'), sort=flop)
).resolve_axis(y="independent",x="independent").resolve_scale(y="independent")

rozdily_21_stoleti = alt.vconcat(top_graf, flop_graf).configure_view(stroke='transparent').configure_axisX(grid=False, domain=False)
rozdily_21_stoleti

alt.VConcatChart(...)

In [31]:
me_to_neurazi(rozdily_21_stoleti, soubor="03_trendy", kredity=kredity['default'])

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/03_trendy.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/03_trendy.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [32]:
podil_zanru("pohádky")

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
6,1989,3115,0.001926,"""pohádky"""
10,1990,3220,0.003106,"""pohádky"""
15,1991,3598,0.004169,"""pohádky"""
52,1992,4452,0.01168,"""pohádky"""
58,1993,5949,0.00975,"""pohádky"""
42,1994,6067,0.006923,"""pohádky"""
56,1995,6784,0.008255,"""pohádky"""
38,1996,7014,0.005418,"""pohádky"""
68,1997,7395,0.009195,"""pohádky"""


In [33]:
podil_zanru("erotick")

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
null,1989,3115,null,"""erotick"""
9,1990,3220,0.002795,"""erotick"""
9,1991,3598,0.002501,"""erotick"""
4,1992,4452,0.000898,"""erotick"""
6,1993,5949,0.001009,"""erotick"""
9,1994,6067,0.001483,"""erotick"""
null,1995,6784,null,"""erotick"""
10,1996,7014,0.001426,"""erotick"""
14,1997,7395,0.001893,"""erotick"""


In [34]:
podil_zanru("dívčí romány")

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
2,1989,3115,0.000642,"""dívčí romány"""
1,1990,3220,0.000311,"""dívčí romány"""
4,1991,3598,0.001112,"""dívčí romány"""
2,1992,4452,0.000449,"""dívčí romány"""
1,1993,5949,0.000168,"""dívčí romány"""
null,1994,6067,null,"""dívčí romány"""
null,1995,6784,null,"""dívčí romány"""
null,1996,7014,null,"""dívčí romány"""
1,1997,7395,0.000135,"""dívčí romány"""


In [35]:
podil_zanru("populárně-naučné publikace")

pocet,rok,len_right,podil,zanr
u32,i64,u32,f64,str
41,1989,3115,0.013162,"""populárně-naučné publikace"""
40,1990,3220,0.012422,"""populárně-naučné publikace"""
52,1991,3598,0.014452,"""populárně-naučné publikace"""
65,1992,4452,0.0146,"""populárně-naučné publikace"""
119,1993,5949,0.020003,"""populárně-naučné publikace"""
131,1994,6067,0.021592,"""populárně-naučné publikace"""
203,1995,6784,0.029923,"""populárně-naučné publikace"""
164,1996,7014,0.023382,"""populárně-naučné publikace"""
209,1997,7395,0.028262,"""populárně-naučné publikace"""


In [36]:
def grafik(z, funkce=podil_zanru):
    return alt.Chart(alt_friendly(funkce(z))).mark_line().encode(
        alt.X("rok:T",axis=alt.Axis(title=None)),
        alt.Y('podil:Q',axis=alt.Axis(
            labelExpr="datum.label * 100 + ' %'", orient='right', domainOpacity=0, tickColor='#DCDDD6',title=None
        ))
    ).configure_view(stroke='transparent')

In [37]:
grafik('erotické')

alt.Chart(...)

In [38]:
grafik('CD-ROM')

alt.Chart(...)

In [39]:
grafik("dvojjazyčná vydání")

alt.Chart(...)

In [40]:
grafik("fantasy romány"	)

alt.Chart(...)

In [41]:
grafik("učebnice základních škol")

alt.Chart(...)

In [42]:
grafik("aforismy")

alt.Chart(...)

In [43]:
grafik("rusk")

alt.Chart(...)

In [44]:
grafik("young adult")

alt.Chart(...)

In [45]:
grafik("příručky")

alt.Chart(...)

In [46]:
grafik("dívčí romány")

alt.Chart(...)

In [47]:
grafik("komiks")

alt.Chart(...)

In [48]:
grafik("detektiv")

alt.Chart(...)

In [49]:
grafik("rozhovory")

alt.Chart(...)

In [50]:
grafik("deníky")

alt.Chart(...)

In [51]:
grafik("autobiogr")

alt.Chart(...)

In [52]:
grafik("encyclopedias")

alt.Chart(...)

In [53]:
grafik("učebnice vysokých škol")

alt.Chart(...)

In [54]:
grafik("kuchař")

alt.Chart(...)

## Kuchařky

In [56]:
kucharky = df.explode("655_a").filter(
        pl.col("655_a").str.contains("kuchař")
    )

In [57]:
kucharky.sample(10)

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,653_a,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Vašák, Jaroslav,""","""jn20000810138""","[""aut""]","""1947-""",null,null,null,null,"""nkc20091852799""",""" nam a22 a 4500""","""090209s2008 xr af e f 0…","""(brož.)""",null,"[""978-80-87156-13-1""]",null,null,null,null,null,null,"""1""","""0""","""Hrnečku, měř!""","""hrníčková kuchařka pro nezkuše…","""Jaroslav Vašák""",null,null,null,null,null,"[""136 s., [16] s. barev. obr. příl. :""]","[""il. ;""]","[""21 cm""]",null,null,null,…,null,"[""7"", ""9""]","""kuchařské recepty""","[""fd132687"", null]","[""czenas"", ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2008,136,"""brožovaná"""
"""1""","""Poštulka, Vladimír,""","""jk01100779""","[""aut""]","""1943-2019""",null,null,null,null,"""cpk20031193949""",""" nam a22 a 4500""","""030121s2002 xr ac e 0…","""(váz.) :""","[""Kč 249,00""]","[""80-903116-7-9""]",null,null,null,null,null,null,"""1""","""0""","""100 nejslavnějších receptů svě…","""(a 101 historka slavných labuž…","""Vladimír Poštulka""",null,null,null,null,null,"[""215 s. :""]","[""barev. il., portréty ;""]","[""22 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","""kuchařské recepty""","[""fd132687"", ""fd133977"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2002,215,"""pevná"""
"""1""","""Sinclair, Mima""","""xx0192804""","[""aut""]",null,null,null,null,null,"""nkc20142607662""",""" cam a22 i 4500""","""150209s2015 xr a e f 0…","""(vázáno)""",null,"[""978-80-249-2592-9""]",null,null,null,null,null,null,"""1""","""0""","""Moučníky z hrníčku""","""40 rychlých moučníků z mikrovl…","""Mima Sinclairová ; z anglickéh…",null,null,null,null,null,"[""95 stran :""]","[""barevné ilustrace ;""]","[""21 cm""]",null,null,null,…,null,"[""7"", ""9""]","""kuchařské recepty""","[""fd132687"", null]","[""czenas"", ""eczenas""]",null,null,null,"[""1""]","[""Somogyi, Viola,""]","[""trl""]","[""1964-""]","[""xx0021382""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2015,95,"""pevná"""
"""1""","""Winnerová, Alena,""","""xx0098336""","[""aut"", ""pht""]","""1980-""",null,null,null,null,"""nkc20152699078""",""" nam a22 i 4500""","""150508s2015 xr a e f 0…","""(brožováno)""",null,"[""978-80-7322-182-9""]",null,null,null,null,null,null,"""1""","""0""","""Špagety""","""Spaghetti : rychlé, chutné a v…","""Alena Winnerová & Josef Winner""",null,null,null,null,null,"[""62 stran :""]","[""barevné ilustrace ;""]","[""24 cm""]",null,null,null,…,null,"[""7"", ""9""]","""kuchařské recepty""","[""fd132687"", null]","[""czenas"", ""eczenas""]",null,null,null,"[""1""]","[""Winner, Josef""]","[""pht""]",null,"[""kv2015874989""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,

In [58]:
def kuchyne(slovo):
    return kucharky.filter(
        pl.col("245_a").str.contains("(?i)" + slovo) | pl.col("245_a").str.contains("(?i)" + slovo)
    ).group_by(
        "rok"
    ).len(
    ).join(kucharky.group_by("rok").len(), on="rok", how="right").with_columns((pl.col("len") / pl.col("len_right")).alias("podil")).sort(by="rok").rename({'len':'pocet'}).fill_null(0)

In [59]:
kuchyne('barbe')

pocet,rok,len_right,podil
u32,i64,u32,f64
0,1989,16,0.0
0,1990,20,0.0
0,1991,39,0.0
0,1992,57,0.0
0,1993,78,0.0
0,1994,80,0.0
0,1995,94,0.0
0,1996,91,0.0
0,1997,95,0.0


In [60]:
grafik('barbe', funkce=kuchyne)

alt.Chart(...)

In [61]:
grafik('vegan', funkce=kuchyne)

alt.Chart(...)

In [62]:
grafik('sous ', funkce=kuchyne)

alt.Chart(...)

In [63]:
grafik('svačin', funkce=kuchyne)

alt.Chart(...)

In [64]:
grafik('pečiv', funkce=kuchyne)

alt.Chart(...)

In [65]:
grafik('chleb', funkce=kuchyne)

alt.Chart(...)

In [66]:
grafik('peče', funkce=kuchyne)

alt.Chart(...)

In [67]:
grafik('diabet', funkce=kuchyne)

alt.Chart(...)

In [68]:
grafik('gril', funkce=kuchyne)

alt.Chart(...)

In [69]:
grafik('cukrov', funkce=kuchyne)

alt.Chart(...)

In [70]:
grafik('zdrav', funkce=kuchyne)

alt.Chart(...)

In [71]:
grafik('bez ', funkce=kuchyne)

alt.Chart(...)

In [72]:
kucharky.filter(pl.col("245_a").str.contains("pro")).with_columns(pl.col("245_a").map_elements(lambda x: x.split("pro")[1].strip())).group_by('245_a').len().sort(by='len',descending=True)

245_a,len
str,u32
"""diabetiky""",25
"""děti""",14
"""každou příležitost""",12
"""každého""",10
"""labužníky""",9
"""moderní ženu""",8
"""každý den""",8
"""cukrářskou výrobu""",8
"""začátečníky""",8


In [73]:
kucharky.filter(pl.col("245_a").str.contains(" s ")).with_columns(pl.col("245_a").map_elements(lambda x: x.split(" s ")[1].split(" ")[0].strip())).group_by('245_a').len().sort(by='len',descending=True)

245_a,len
str,u32
"""láskou""",10
"""Měsícem""",6
"""Ellou""",5
"""konopím""",4
"""fantazií""",4
"""Habadějem""",4
"""bylinkami""",4
"""dětmi""",4
"""dělenou""",3


In [74]:
kucharky.filter(pl.col("245_a").str.contains(" bez ")).with_columns(pl.col("245_a").map_elements(lambda x: x.split(" bez ")[1].split(" ")[0].strip())).group_by('245_a').len().sort(by='len',descending=True)

245_a,len
str,u32
"""lepku""",24
"""cukru""",6
"""mléka""",6
"""cholesterolu""",6
"""vážení""",5
"""hladovění""",3
"""servítků""",2
"""lepku,""",2
"""zkušeností""",2


In [75]:
grafik(' s ', funkce=kuchyne)

alt.Chart(...)

In [76]:
grafik(' bez ', funkce=kuchyne)

alt.Chart(...)

In [77]:
grafik('frit', funkce=kuchyne)

alt.Chart(...)

In [78]:
grafik('indi', funkce=kuchyne)

alt.Chart(...)

In [79]:
grafik('japon', funkce=kuchyne)

alt.Chart(...)

In [80]:
grafik('vietn', funkce=kuchyne)

alt.Chart(...)

In [81]:
grafik('pán[ev]', funkce=kuchyne)

alt.Chart(...)

In [82]:
grafik('mikrov', funkce=kuchyne)

alt.Chart(...)

In [83]:
grafik('gril', funkce=kuchyne)

alt.Chart(...)

In [84]:
grafik('sex', funkce=kuchyne)

alt.Chart(...)

In [85]:
grafik('babi', funkce=kuchyne)

alt.Chart(...)

In [86]:
grafik('pomaz', funkce=kuchyne)

alt.Chart(...)

In [87]:
grafik('cukr', funkce=kuchyne)

alt.Chart(...)

In [88]:
grafik('hrní', funkce=kuchyne)

alt.Chart(...)

In [89]:
grafik('hubn', funkce=kuchyne)

alt.Chart(...)

In [90]:
grafik('omáč', funkce=kuchyne)

alt.Chart(...)

In [91]:
grafik('pol[íé]v', funkce=kuchyne)

alt.Chart(...)

In [92]:
grafik('pomaz', funkce=kuchyne)

alt.Chart(...)

In [93]:
grafik('keto', funkce=kuchyne)

alt.Chart(...)

In [94]:
grafik('paleo', funkce=kuchyne)

alt.Chart(...)

In [95]:
grafik('houb', funkce=kuchyne)

alt.Chart(...)

In [96]:
grafik('(kvas|kvaš|ferment)', funkce=kuchyne)

alt.Chart(...)

In [97]:
grafik('makrobiot', funkce=kuchyne)

alt.Chart(...)

In [98]:
grafik('bílkov', funkce=kuchyne)

alt.Chart(...)

In [99]:
kucharky.select(pl.col("245_a")).to_series().to_list()

['Pečeme s konopím',
 'Asijská kuchařka',
 'Byliny na srdce, cévy, krev',
 'Neodolatelné nanuky',
 'Svět v kuchyni',
 'Vaří Čirina',
 'Po čem skutečně hladovíme?',
 'Vitaminy v kuchyni',
 'Péct dokáže každý',
 'Vaříme z exotických a léčivých hub',
 'Těstoviny a rýže levně a rychle',
 'Sýrová kuchařka',
 'Polévky našich babiček',
 'Kouzlo kvásku',
 'Pěstujeme bylinky',
 'Pampelišky k jídlu',
 'Slavnostní kuchařka',
 'Cukrářské práce',
 'Už se vaří, má panenko',
 'Sugar-free kuchyně',
 'Dnes vařím já',
 'Bezlepkové pečení pro každého',
 'Tajné bleskové recepty naší babičky',
 'Babicovy dobroty',
 'Oblíbené saláty',
 'Autoimunitní řešení',
 'Vaříme ryby v Norsku',
 'Gastrokroužek',
 'Velikonoce nejen v kuchyni',
 'Sladkosti bez výčitek',
 'Osladíme si život',
 'Mileniny recepty',
 'Vaříme z těstovin a rýže',
 'Na grilu',
 'Variace na pštrosí téma',
 'Fit rodinka',
 'Neobvyklé recepty sladkostí',
 'Vejce nebo slepice',
 'Dětská party',
 'Užitkové rostliny, aneb, Co pěstovat, sbírat a jíst 

## Komiksy

In [212]:
df.filter(pl.col("245_a").str.contains("Naruto"))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,653_a,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Kishimoto, Masashi,""","""xx0137635""","[""aut""]","""1974-""",null,null,null,null,"""nkc20162837349""",""" nam a22 i 4500""","""161010s2016 xr a d 6 0…","""(brožováno) :""","[""Kč 189,00""]","[""978-80-7449-408-6""]",null,null,null,null,null,null,"""1""","""0""","""Naruto""",null,"""Masaši Kišimoto ; překlad: Jan…","[""30. díl,""]","""Sakura a Babi Čijo""",null,null,null,"[""186 stran :""]","[""ilustrace ;""]","[""18 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","[""manga"", ""akční a dobrodružné komiksy"", … ""juvenile literature""]","[""fd1057492"", ""fd1057471"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"[""1""]","[""Horgoš, Jan""]","[""trl""]",null,"[""xx0098660""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2016,186,"""brožovaná"""
"""1""","""Kishimoto, Masashi,""","""xx0137635""","[""aut"", ""ill""]","""1974-""",null,null,null,null,"""nkc20162791679""",""" nam a22 i 4500""","""160408s2016 xr a d 6 0…","""(brožováno) :""","[""Kč 189,00""]","[""978-80-7449-373-7""]",null,null,null,null,null,null,"""1""","""0""","""Naruto""",null,"""Masaši Kišimoto ; překlad: Jan…","[""27. díl,""]","""Vzhůru na cesty""",null,null,null,"[""185 stran :""]","[""ilustrace ;""]","[""18 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","[""manga"", ""akční a dobrodružné komiksy"", … ""juvenile literature""]","[""fd1057492"", ""fd1057471"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"[""1""]","[""Horgoš, Jan""]","[""trl""]",null,"[""xx0098660""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2016,185,"""brožovaná"""
"""1""","""Kishimoto, Masashi,""","""xx0137635""","[""aut"", ""ill""]","""1974-""",null,null,null,null,"""nkc20203262303""",""" nam a22 i 4500""","""201104s2020 xr a d 6 0…","""(brožováno) :""","[""Kč 189,00""]","[""978-80-7449-904-3""]",null,null,null,null,null,null,"""1""","""0""","""Naruto""",null,"""Masaši Kišimoto ; překlad: Jan…","[""48. díl,""]","""Slavící vesnice!!""",null,null,null,"[""204 stran :""]","[""ilustrace ;""]","[""18 cm""]",null,null,null,…,null,"[""7"", ""7"", … ""9""]","[""manga"", ""akční a dobrodružné komiksy"", … ""juvenile literature""]","[""fd1057492"", ""fd1057471"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"[""1""]","[""Horgoš, Jan""]","[""trl""]",null,"[""xx0098660""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2020,204,"""brožovaná"""
"""1""","""Kishimoto, Masashi,""","""xx0137635""","[""aut"", ""ill""]","""1974-""",null,null,null,null,"""nkc20112252941""",""" nam a22 a 4500""","""111118s2011 xr a d 6 0…","""(brož.) :""","[""Kč 179,00""]","[""978-80-7449-066-8""]",null,null,null,null,null,null,"""1""","""0""","""Naruto""",null,"""[scénář a kresba] Masaši Kišim…","[""3. díl,""]","""Pro své sny""",null,null,null

In [214]:
df.explode("655_a").filter(
        pl.col("655_a").str.contains("komiks")
    ).sort("rok").unique(subset=['rok','100_a','245_a','245_p']).group_by('rok').len().sort(by="rok")

rok,len
i64,u32
1989,4
1990,6
1991,7
1992,8
1993,5
1994,11
1995,8
1996,6
1997,10


In [210]:
df.explode("655_a").filter(
        pl.col("655_a").str.contains("komiks")
    ).sort("rok").unique(subset=['rok','100_a','245_a']).group_by('rok').len().sort(by="rok")

rok,len
i64,u32
1989,4
1990,6
1991,7
1992,8
1993,5
1994,11
1995,8
1996,6
1997,10


In [220]:
df.filter(pl.col("rok") == 2024).explode("655_a").filter(
        pl.col("655_a").str.contains("komiks")
    ).group_by("245_a").agg(pl.col("245_p").unique().len()).sort(by="245_p",descending=True)

245_a,245_p
str,u32
"""Zabiják démonů""",5
"""One piece""",5
"""Ú.P.V.O. - peklo na zemi""",4
"""Chainsaw man""",4
"""Conan""",4
"""Ježek Sonic""",4
"""Bleach""",4
"""Naruto""",4
"""Jujutsu kaisen""",4
